In [1]:
%matplotlib qt

In [2]:
import matplotlib as mpl
import sys
sys.path.append("/Users/yao/Smilei")
import happi
import numpy as np
import matplotlib.pyplot as plt

jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd
jet_vals = jetcmap(np.arange(9)) #extract those values as an array 
jet_vals[0] = [1.0, 1, 1.0, 1] #change the first value 
jet_vals[8] = [0.0, 0, 0.0, 1] #change the first value 
newcmap = mpl.colors.LinearSegmentedColormap.from_list("mine", jet_vals) 

from matplotlib import font_manager

font_dirs = ['/Users/yao/Documents/Calibri and Cambria Fonts/']
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

# set font
plt.rcParams['font.family'] = 'Calibri'

plt.rc('text', usetex=False)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('axes', labelsize=14)
plt.rc('legend', fontsize=12)

/var/folders/2t/97rc3fl92tg15k2l_4sk5hsh0000gn/T/ipykernel_11030/621321092.py:8: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd


In [3]:
## prepare simulation data

wkdir = [
    #  '/Users/yao/Documents/Data/IFE/test0/laser_propagation_3d', # with only test particles
    #  '/Users/yao/Documents/Data/IFE/test0/laser_propagation_3d_target/', # with a thin foil target
    #  '/Users/yao/Desktop/test1/',
    #  '/Users/yao/Desktop/IFE_FlatTop_3D/',
    #  '/Users/yao/Desktop/ife_yao0/',
    #  '/Users/yao/Desktop/data/ife_yao2_cp2/',
    #  '/Users/yao/Desktop/data/ife_yao2_cp3/',
    #  '/Users/yao/Desktop/ife_yao3/',
    #  '/Users/yao/Documents/Data/IFE/ife_yao1_Ly25.6/',
    '/Users/yao/Documents/Data/IFE/ife_yao4_benchmark_probe_reso/',  # a0=225, tp=5t0, ne=60nc
    '/Users/yao/Documents/Data/IFE/ife_yao6/', # a0=300, tp=5t0, ne=60nc
    '/Users/yao/Documents/Data/IFE/ife_yao9/', # a0=325, tp=5t0, ne=60nc
    '/Users/yao/Documents/Data/IFE/ife_yao7/', # a0=350, tp=5t0, ne=60nc
    '/Users/yao/Documents/Data/IFE/ife_yao10/',# a0=375, tp=5t0, ne=60nc
    '/Users/yao/Documents/Data/IFE/ife_yao8/', # a0=400, tp=5t0, ne=60nc
    '/Users/yao/Documents/Data/IFE/ife_yao5/', # a0=450, tp=5t0, ne=60nc
    '/Users/yao/Documents/Data/IFE/ife_yao11/', # a0=225, tp=50t0, ne=60nc
    '/Users/yao/Documents/Data/IFE/ife_yao12/', # a0=225, tp=5t0, ne=15nc
    ]

S0 = happi.Open(wkdir[0], reference_angular_frequency_SI = 2.0*np.pi*3e8/(1.0*1e-6))
S1 = happi.Open(wkdir[1], reference_angular_frequency_SI = 2.0*np.pi*3e8/(1.0*1e-6))
S2 = happi.Open(wkdir[2], reference_angular_frequency_SI = 2.0*np.pi*3e8/(1.0*1e-6))
S3 = happi.Open(wkdir[3], reference_angular_frequency_SI = 2.0*np.pi*3e8/(1.0*1e-6))
S4 = happi.Open(wkdir[4], reference_angular_frequency_SI = 2.0*np.pi*3e8/(1.0*1e-6))
S5 = happi.Open(wkdir[5], reference_angular_frequency_SI = 2.0*np.pi*3e8/(1.0*1e-6))
S6 = happi.Open(wkdir[6], reference_angular_frequency_SI = 2.0*np.pi*3e8/(1.0*1e-6))
S7 = happi.Open(wkdir[7], reference_angular_frequency_SI = 2.0*np.pi*3e8/(1.0*1e-6))
S8 = happi.Open(wkdir[8], reference_angular_frequency_SI = 2.0*np.pi*3e8/(1.0*1e-6))



18790481920.0
Loaded simulation '/Users/yao/Documents/Data/IFE/ife_yao4_benchmark_probe_reso/'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics
Scanning for new particle diagnostics
18790481920.0
Loaded simulation '/Users/yao/Documents/Data/IFE/ife_yao6/'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics
Scanning for new particle diagnostics
18790481920.0
Loaded simulation '/Users/yao/Documents/Data/IFE/ife_yao9/'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning 

In [4]:
Lx = S0.namelist.Lx / 2 / np.pi      # in um
print("Lx = ", Lx)
Ly = S0.namelist.Ly / 2 / np.pi      # in um
print("Ly = ", Ly)
Lz = S0.namelist.Lz / 2 / np.pi      # in um
print("Lz = ", Lz)

Lx =  80.0
Ly =  51.2
Lz =  51.2


In [ ]:
Bxm0 = S7.Field(0,"Bx_m",subset={'z':Lz/2.},
         units=['um','fs','T'],
         vsym=True,
         cmap='smileiD',
         aspect='equal',
         )
# Bxm.set(layout='tight')
Bxm1 = S8.Field(0,"Bx_m",subset={'z':Lz/2.},
         units=['um','fs','T'],
         vsym=True,
         cmap='smileiD',
         aspect='equal',
         )

happi.multiSlide(Bxm0, Bxm1,shape=(2,1))

In [43]:
def fft_field_analysis(S, diag_num, field_name, slice_direction, slice_position, shift, timestep, k0x, k0y, wx, wy):
    """
    Perform FFT analysis on a given field from the simulation data.
    
    Parameters:
    S : happi.Simulation
        The simulation object.
    diag_num : int
        The diagnostic number of the field.
    field_name : str
        The name of the field to analyze.
    slice_direction : str
        e.g., 'x', 'y', 'z', etc.
    slice_position : float
        The position to slice the field (e.g., Lz/2).
    shift : bool
        Whether to shift the sample region to avoid boundary condition issues.
    timestep : int
        The timestep at which to perform the analysis.
    k_filter_x, k_filter_y : float
        The mask to filter out the signals. Laser is 1.
    
    Returns:
    kx : np.ndarray
        The wave numbers in the x-direction.
    ky : np.ndarray
        The wave numbers in the y-direction.
    spectrum : np.ndarray
        The FFT spectrum of the field.
    """
    
    twopi = 2.0*np.pi
    var0_time = S.Probe(diag_num, field_name, units=['fs']).getTimesteps()
    print('timesteps of var : ', var0_time)
    print('num. of timesteps: ', var0_time.size)
    
    var0 = S.Probe(diag_num, field_name, units=['fs']).getData(timestep=var0_time[timestep])[0]
    print('shape of var: ', var0.shape)
    
    var0_time_real = S.Probe(diag_num, field_name, units=['fs']).getTimes()
    print('time of var : {:.2f} fs'.format(var0_time_real[timestep]))

    field = S.Probe(diag_num, field_name, 
                    subset={slice_direction:slice_position},
                    units=['um','fs','1e9 Gauss'],
                    vsym=True)
    
    data = np.array(field.getData()[timestep])

    if shift==True:
        nx = data.shape[0]
        ny = data.shape[1]
        sf = 0 # shift of cells to get rid of boundary condition
        print('note that the sample region is shifted to get rid of the BC issue')
        print('No. of cells being shifted, ', sf)
#         print(nx, ny)
        data = data[sf:(nx-sf),sf:(ny-sf)]


    field_x = np.array(field.getAxis('axis1'))
    field_y = np.array(field.getAxis('axis2'))
    
    Nx = data.shape[0]
    Ny = data.shape[1]
    
    dx = (field_x[1] - field_x[0])[0]
    dy = (field_y[1] - field_y[0])[1]

    # kx = np.fft.fftfreq(Nx, d=dx) * 2 * np.pi #spatial freq extents
    # ky = np.fft.fftfreq(Ny, d=dy) * 2 * np.pi
    # KX, KY = np.meshgrid(kx, ky, indexing='ij')

    kx=np.linspace(-Nx/2,Nx/2-1,Nx)/((Nx)*dx)  ## -1? min&max??
    ky=np.linspace(-Ny/2,Ny/2-1,Ny)/((Ny)*dy)
    KX, KY = np.meshgrid(twopi*kx, twopi*ky, indexing='ij')
    
    # construct mask

    # super gaussian mask  
    # pwr=12 #super gaussian power
    # mask = np.exp(- np.sqrt((KX / k_filter_x)**2 + (KY / k_filter_y)**2)**pwr)
    # plt.imshow(mask.T, extent=(kx.min(), kx.max(), ky.min(), ky.max()), origin='lower')

    def notch_mask(KX, KY, k0x, k0y, wx, wy, p=12):
        # four symmetric notches around ±k0x and ±k0y
        notch_1 = np.exp(-(((KX - k0x)/wx)**2 + ((KY - k0y)/wy)**2)**(p/2))
        notch_2 = np.exp(-(((KX + k0x)/wx)**2 + ((KY - k0y)/wy)**2)**(p/2))
        notch_3 = np.exp(-(((KX - k0x)/wx)**2 + ((KY + k0y)/wy)**2)**(p/2))
        notch_4 = np.exp(-(((KX + k0x)/wx)**2 + ((KY + k0y)/wy)**2)**(p/2))
        return (1 - notch_1) * (1 - notch_2) * (1 - notch_3) * (1 - notch_4)
    mask = notch_mask(KX, KY, k0x, k0y, wx, wy, p=12)
    # plt.imshow(mask.T, extent=(kx.min(), kx.max(), ky.min(), ky.max()), origin='lower')
    # plt.xlim([0,1])
    # plt.ylim([0,1])

    
    # data_fft  = np.fft.fft2(data, axes=(0,1))   # shape (Nx, Ny)
    # Diag_data = np.real(np.fft.ifft2(data_fft* mask[:,:]))

    data_fft  = np.fft.fft2(data, axes=(0,1))#/Nx/Ny*2   # shape (Nx, Ny)
    Diag_data = np.real(np.fft.ifft2(data_fft* mask[:,:]))#*Nx*Ny/2
    
    data_fft_shift = np.fft.fftshift(data_fft)
    data_fft_shift_masked = data_fft_shift* mask[:,:]
    Diag_data_shift_back = np.fft.ifftshift(data_fft_shift* mask[:,:])
    # Diag_data = np.real(np.fft.ifft2(Diag_data_shift_back))
    # Diag_data = np.real(np.fft.ifft2(data_fft_shift* mask[:,:]))
    
    return kx, ky, data, data_fft, data_fft_shift, data_fft_shift_masked, Diag_data


In [ ]:
k_filter_x = 0.3
k_filter_y = 15

kx0, ky0, Bx0, Bx0_fft, Bx0_masked = fft_field_analysis(S0, 0, "Bx", 'z', Lz/2., -1, k_filter_x, k_filter_y)
kx1, ky1, Bx1, Bx1_fft, Bx1_masked = fft_field_analysis(S1, 0, "Bx", 'z', Lz/2., -1, k_filter_x, k_filter_y)
kx2, ky2, Bx2, Bx2_fft, Bx2_masked = fft_field_analysis(S2, 0, "Bx", 'z', Lz/2., -1, k_filter_x, k_filter_y)
kx3, ky3, Bx3, Bx3_fft, Bx3_masked = fft_field_analysis(S3, 0, "Bx", 'z', Lz/2., -1, k_filter_x, k_filter_y)
kx4, ky4, Bx4, Bx4_fft, Bx4_masked = fft_field_analysis(S4, 0, "Bx", 'z', Lz/2., -1, k_filter_x, k_filter_y)
kx5, ky5, Bx5, Bx5_fft, Bx5_masked = fft_field_analysis(S5, 0, "Bx", 'z', Lz/2., -1, k_filter_x, k_filter_y)
kx6, ky6, Bx6, Bx6_fft, Bx6_masked = fft_field_analysis(S6, 0, "Bx", 'z', Lz/2., -1, k_filter_x, k_filter_y)
kx7, ky7, Bx7, Bx7_fft, Bx7_masked = fft_field_analysis(S7, 0, "Bx", 'z', Lz/2., -1, k_filter_x, k_filter_y)
kx8, ky8, Bx8, Bx8_fft, Bx8_masked = fft_field_analysis(S8, 0, "Bx", 'z', Lz/2., -1, k_filter_x, k_filter_y)

In [49]:
twopi = 1.0
# k0x = 0.35*twopi
# k0y = 0.33*twopi
# wx  = 0.5*twopi
# wy  = 0.7*twopi
k0x = 2.0*twopi
k0y = 2.3*twopi
wx  = 3.5*twopi
wy  = 13*twopi
# kx0, ky0, Bx0, Bx0_fft, Bx0_fft_shifted, Bx0_masked = fft_field_analysis(S0, 0, "Bx", 'z', Lz/2., True, -1, k_filter_x, k_filter_y)
kx3, ky3, Bx3, Bx3_fft, Bx3_fft_shifted, Bx3_fft_shifted_masked, Bx3_masked = fft_field_analysis(S3, 0, "Bx", 'z', Lz/2., True, -1, k0x, k0y, wx, wy)


timesteps of var :  [   0.  182.  364.  546.  728.  910. 1092. 1274. 1456. 1638. 1820. 2002.
 2184. 2366. 2548. 2730. 2912. 3094. 3276.]
num. of timesteps:  19
shape of var:  (1000, 1220)
time of var : 299.47 fs
note that the sample region is shifted to get rid of the BC issue
No. of cells being shifted,  0


In [7]:
plt.imshow(np.log10(np.abs(Bx0_fft.T)), 
           extent=(kx0.min()/2/np.pi, kx0.max()/2/np.pi, ky0.min()/2/np.pi, ky0.max()/2/np.pi), 
           origin='lower', 
           aspect='auto')
plt.colorbar(label='Log10 |Bx0 FFT|')
# plt.xlim([-5,5])
# plt.ylim([-5,5])
plt.xlabel('kx (1/um)')
plt.ylabel('ky (1/um)')
plt.title('FFT of Bx field at z=Lz/2, timestep=10')

NameError: name 'Bx0_fft' is not defined

In [50]:
plt.figure()
plt.imshow(
            # np.log10(np.abs(Bx3_fft_shifted.T)), 
            np.abs(Bx3_fft_shifted_masked.T),
           extent=(kx3.min(), kx3.max(), ky3.min(), ky3.max()), 
           origin='lower', 
        #    vmin=-5,
        #    vmax=-1,
           aspect='auto')
plt.colorbar(label='|Bx FFT|')
plt.xlim([-1,1])
plt.ylim([-1,1])
plt.xlabel('kx (1/um)')
plt.ylabel('ky (1/um)')
# plt.title('FFT of Bx field at z=Lz/2, time=133.10 fs')
plt.tight_layout()

In [51]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(figsize=(6, 5))

im = ax.imshow(
               Bx3_masked.T,
               # Bx3.T,
               origin='lower',
               extent=[0, Lx, 0, Ly],
               vmin=-10, vmax=10,
               aspect='equal',
               cmap='bwr')

# Make a colorbar axis that exactly matches the height of `ax`
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)  # width & gap
cb = fig.colorbar(im, cax=cax,label='Bx (GG)')

ax.set_xlabel('x (um)')
ax.set_ylabel('y (um)')
fig.tight_layout()

# plt.tight_layout()
# plt.savefig('/Users/yao/Desktop/Bx_a0_225.png', dpi=300, bbox_inches='tight')

In [47]:
Bx3_masked.max(), Bx3_masked.min()

(21.003069547455805, -15.099125334440206)

In [48]:
Bx3.max(), Bx3.min()

(21.003015518188477, -15.098770141601562)

In [164]:
plt.plot(kx0, (np.abs(Bx0_fft_shifted[:,589].T)))
plt.xlabel('kx (1/um)')
plt.ylabel('Log10 |Bx FFT| at ky=0')
plt.title('Lineout of FFT of Bx field at z=Lz/2, time=133.10 fs, ky=0')

Text(0.5, 1.0, 'Lineout of FFT of Bx field at z=Lz/2, time=133.10 fs, ky=0')

In [114]:
S0.Probe(0, "Bx", 
        #  subset={'z':Lz/2.},
         units=['um','fs','1e9 Gauss'],
         vsym=1,
         aspect='equal',
         ).slide()

Probe #0: 2-dimensional, with fields Bx,Jy,Jz,Jx,By,Bz,Ey,Ez,Ex
	p0 = 0.0 0.0 160.8495438637974
	p1 = 502.6548245743669 0.0 160.8495438637974
	p2 = 0.0 321.6990877275948 160.8495438637974
	number = 1000 1220


In [61]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(figsize=(6, 5))

im = ax.imshow(Bx0_masked.T,
               origin='lower',
               extent=[0, Lx, 0, Ly],
               vmin=-2, vmax=2,
               aspect='equal',
               cmap='bwr')

# Make a colorbar axis that exactly matches the height of `ax`
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)  # width & gap
cb = fig.colorbar(im, cax=cax,label='Bx (GG)')

ax.set_xlabel('x (um)')
ax.set_ylabel('y (um)')

# plt.tight_layout()
plt.savefig('/Users/yao/Desktop/Bx_a0_225.png', dpi=300, bbox_inches='tight')

In [232]:
import scienceplots as splt
plt.style.use(['nature', 'notebook', 'grid','high-vis'])


In [ ]:

cell_y = 30 # 1 cell = 0.04 um along y
center = int(Bx0_masked.shape[1] / 2)
xx = np.linspace(0, Lx, Bx0_masked.shape[0])
yy = np.linspace(0, Ly, Bx0_masked.shape[1])
plt.plot(xx, np.average(Bx0_masked[:,(center-cell_y):(center+cell_y)], axis=1), label='low-reso', color='blue', linestyle='-')
cell_y = 60 # 1 cell = 0.04 um along y
center = int(Bx1_masked.shape[1] / 2)
xx = np.linspace(0, Lx, Bx1_masked.shape[0])
yy = np.linspace(0, Ly, Bx1_masked.shape[1])
plt.plot(xx, np.average(Bx1_masked[:,(center-cell_y):(center+cell_y)], axis=1), label='high-reso', color='red', linestyle='--')
plt.xlabel('x (um)')
plt.ylabel('Bx (T)')
plt.title('Averaged Bx field at y = {:.1f} um with a width of {:.1f} um'.format(yy[center], 2*cell_y*(Ly/Bx1_masked.shape[1])))
plt.grid(alpha=0.3, linestyle=':', which='both')
plt.tight_layout()
plt.xlim(0, Lx)
plt.legend()
# plt.ylim(-5000, 35000)
# plt.savefig('/Users/yao/Desktop/Bx_profile_x_benchmark.png', dpi=300)

In [54]:
cell_y = 60 # 1 cell = 0.04 um along y
center = int(Bx0_masked.shape[1] / 2)
xx = np.linspace(0, Lx, Bx0_masked.shape[0])
yy = np.linspace(0, Ly, Bx0_masked.shape[1])
plt.plot(xx, np.average(Bx0_masked[:,(center-cell_y):(center+cell_y)], axis=1), label='a=225', color='blue', linestyle='-')
plt.plot(xx, np.average(Bx1_masked[:,(center-cell_y):(center+cell_y)], axis=1), label='a=300', color='cyan', linestyle='-')
plt.plot(xx, np.average(Bx2_masked[:,(center-cell_y):(center+cell_y)], axis=1), label='a=350', color='green', linestyle='-')
plt.plot(xx, np.average(Bx3_masked[:,(center-cell_y):(center+cell_y)], axis=1), label='a=400', color='yellow', linestyle='-')
plt.plot(xx, np.average(Bx4_masked[:,(center-cell_y):(center+cell_y)], axis=1), label='a=450', color='red', linestyle='--')

plt.xlabel('x (um)')
plt.ylabel('Bx (GG)')
plt.title('Averaged Bx field at y = {:.1f} um with a width of {:.1f} um'.format(yy[center], 2*cell_y*(Ly/Bx0_masked.shape[1])))
plt.grid(alpha=0.3, linestyle=':', which='both')
plt.tight_layout()
plt.xlim(0, Lx)
plt.legend()
# plt.ylim(-5000, 35000)
plt.savefig('/Users/yao/Desktop/Bx_profile_x.png', dpi=300)
print(np.abs(np.average(Bx0_masked[:,(center-cell_y):(center+cell_y)], axis=1)).max())
print(np.abs(np.average(Bx1_masked[:,(center-cell_y):(center+cell_y)], axis=1)).max())
print(np.abs(np.average(Bx2_masked[:,(center-cell_y):(center+cell_y)], axis=1)).max())
print(np.abs(np.average(Bx3_masked[:,(center-cell_y):(center+cell_y)], axis=1)).max())
print(np.abs(np.average(Bx4_masked[:,(center-cell_y):(center+cell_y)], axis=1)).max())

0.16925295785120995
0.36818620802685226
1.7789168661190833
1.5991636719694646
1.8498827572841303


In [144]:
cell_x = 80 # 1 cell = 0.08 um along x
target_surface = int(Bx0_masked.shape[0] * 0.45) # target is at 0.45 Lx
plt.plot(yy, np.average(Bx0_masked[(target_surface-cell_x):(target_surface+cell_x), :], axis=0))
plt.xlabel('y (um)')
plt.ylabel('Bx (T)')
plt.title('Averaged Bx field at x = {:.1f} um with a width of {:.1f} um'.format(xx[target_surface], 2*cell_x*(Lx/Bx0_masked.shape[0])))
plt.grid(alpha=0.3, linestyle=':', which='both')
plt.tight_layout()
plt.xlim(0, Ly)
# plt.ylim(-5000, 35000)
plt.savefig('/Users/yao/Desktop/Bx_profile_y.png', dpi=300)

In [53]:
import numpy as np
import matplotlib.pyplot as plt

# ---- Fine-tuned data (from the image) ----
# a0 values (x-axis) and corresponding Bx in GGauss (y-axis)
a0 = np.array([280, 300, 325, 350, 375, 400, 500], dtype=float)
Bx = np.array([0.28, 0.53, 0.78, 1.27, 1.60, 2.05, 3.30], dtype=float)

a1 = np.array([  225,  300,  350,  400,  450], dtype=float)
# Bx1 = np.array([0.46, 0.38, 3.14, 1.88, 2.21], dtype=float) # filter: 0.5, 15
Bx1 = np.array([0.17, 0.37, 1.77, 1.60, 1.85], dtype=float) # filter: 0.3, 1.5


# ---- Fit a power law: Bx = C * a0^p ----
log_a0 = np.log(a0)
log_Bx = np.log(Bx)
p, logC = np.polyfit(log_a0, log_Bx, 1)
C = np.exp(logC)

# Smooth curve for the fitted relation
a0_fit = np.linspace(200, 520, 300)
Bx_fit = C * a0_fit**p

# ---- Plot ----
plt.figure(figsize=(5.2, 4.6))
plt.scatter(a0, Bx, s=120, color='b', label='former 3D PIC data')              # default color, similar to the figure
plt.plot(a0_fit, Bx_fit, '--', lw=1.6, color='b', label="fitting")  # dashed fit line
plt.scatter(a1, Bx1, s=100, color='red', marker='x', label='new 2D PIC data')  # new data points

# Axes style to match the original figure
plt.xlabel(r'$a_0$', fontsize=14)
plt.ylabel(r'$B_x$ [GGauss]', fontsize=14)
plt.xlim(200, 520)
plt.ylim(0.0, 6.0)
plt.tight_layout()
plt.legend()
plt.title(f"Fitted power law: Bx = {C:.3e} * a0^{p:.2f}")
plt.show()

# Print the fitted law for reference
# print(f"Fitted power law: Bx = {C:.3e} * a0^{p:.2f}")
plt.savefig('/Users/yao/Desktop/scaling.png', dpi=300, bbox_inches='tight')

2025-10-29 13:14:07.448 Python[2197:4019527] error messaging the mach port for IMKCFRunLoopWakeUpReliable


In [32]:
S0.ParticleBinning(0,units=['um','fs','T','V/m','cm^-3'],
                   aspect='equal',
                #    data_log=True,
                   cmap='smileiD',
                   vsym=True,
                #    vmin=19,
                #    vmax=23,
                #    subset={'z':S0.namelist.Ltrans/2.},
                #    subset={'x':S0.namelist.Ltrans/2.},
                   ).slide()    


#0 - user_function of species # 1
    y from 0 to 160.85 in 128 steps 
    z from 0 to 160.85 in 128 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size and by grid_length[0]



In [33]:
S0.ParticleBinning(1,units=['um','fs','T','V/m','cm^-3'],
                   aspect='equal',
                #    data_log=True,
                   cmap='smileiD',
                   vsym=True,
                #    vmin=19,
                #    vmax=23,
                #    subset={'z':S0.namelist.Ltrans/2.},
                #    subset={'x':S0.namelist.Ltrans/2.},
                   ).slide()    


#1 - user_function of species # 0
    y from 0 to 160.85 in 128 steps 
    z from 0 to 160.85 in 128 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size and by grid_length[0]



In [34]:
S0.ParticleBinning(2,units=['um','fs','T','V/m','cm^-3'],
                   aspect='equal',
                #    data_log=True,
                   cmap='smileiD',
                   vsym=True,
                #    vmin=19,
                #    vmax=23,
                #    subset={'z':S0.namelist.Ltrans/2.},
                #    subset={'x':S0.namelist.Ltrans/2.},
                   ).slide()    


#2 - user_function of species # 2
    y from 0 to 160.85 in 128 steps 
    z from 0 to 160.85 in 128 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size and by grid_length[0]



In [43]:
S0.ParticleBinning(3,units=['um','fs','T','V/m','cm^-3'],
                   aspect='equal',
                   data_log=True,
                   cmap=newcmap,
                #    vsym=True,
                   vmin=20,
                   vmax=24,
                   subset={'z':S0.namelist.Lz/2.},
                #    subset={'x':S0.namelist.Ltrans/2.},
                   ).slide()  


#3 - Number density of species # 1
    x from 0 to 502.655 in 1000 steps 
    y from 0 to 160.85 in 620 steps 
    z from 79.7965 to 81.0531 in 3 steps 

subset at z = [80.4248] L_r
The value in each bin is the sum of the `deposited_quantity` divided by the bin size



/Users/yao/Smilei/happi/_Diagnostics/Diagnostic.py:857: RuntimeWarning: divide by zero encountered in log10
  return self._np.log10( self._dataLinAtTime(t) )


In [35]:
S1.Field(0,"Ez",
               subset={'z':Lz},
               # average={'z':Lz/2.},
         units=['um','fs','V/m'],
         vsym=True,
         cmap='smileiD',
         aspect='equal',
         ).slide()

Field diagnostic #0: Ez
	Grid spacing: 1.2566370614359172, 1.2566370614359172, 1.2566370614359172
	subset at z = [25.13274123] L_r


In [45]:
Ey0 = S0.Probe(0,"Bx",
               subset={'z':Lz/2.},
               # average={'z':Lz/2.},
         units=['um','fs','V/m','T'],
         vsym=True,
         cmap='smileiD',
         aspect='equal',
         ).slide()

# Ey1 = S1.Probe(0,"Ey",
#                subset={'z':Lz},
#                # average={'z':Lz/2.},
#          units=['um','fs','V/m','T'],
#          vsym=True,
#          cmap='smileiD',
#          aspect='auto',
#          )#.slide()

# happi.multiSlide(Ey0, Ey1,shape=(2,1))

Probe #0: 2-dimensional, with fields Bx,Jy,Jz,Jx
	p0 = 0.0 0.0 80.4247719318987
	p1 = 502.6548245743669 0.0 80.4247719318987
	p2 = 0.0 160.8495438637974 80.4247719318987
	number = 1000 620


In [225]:
def fft_field_analysis2(S, diag_num, field_name, slice_direction, slice_position, timestep, k_filter_x, k_filter_y):
    """
    Perform FFT analysis on a given field from the simulation data.
    
    Parameters:
    S : happi.Simulation
        The simulation object.
    diag_num : int
        The diagnostic number of the field.
    field_name : str
        The name of the field to analyze.
    slice_direction : str
        e.g., 'x', 'y', 'z', etc.
    slice_position : float
        The position to slice the field (e.g., Lz/2).
    timestep : int
        The timestep at which to perform the analysis.
    k_filter_x, k_filter_y : float
        The mask to filter out the signals. Laser is 1.
    
    Returns:
    kx : np.ndarray
        The wave numbers in the x-direction.
    ky : np.ndarray
        The wave numbers in the y-direction.
    spectrum : np.ndarray
        The FFT spectrum of the field.
    """
    field = S.Probe(diag_num, field_name, subset={slice_direction:slice_position},
                    units=['um','fs','T'],
                    vsym=True)
    
    data = np.array(field.getData()[timestep])
    field_x = np.array(field.getAxis('axis1')[:,0])
    field_y = np.array(field.getAxis('axis2')[:,1])
    print(field_x)
    # print('shape of field_x: ', field_x.shape)
    Nx = data.shape[0]
    print('Nx: ', Nx)
    Ny = data.shape[1]
    dx = (field_x[1] - field_x[0])
    dy = (field_y[1] - field_y[0])
    kx = np.fft.fftfreq(len(field_x), d=dx) #* 2 * np.pi #spatial freq extents
    ky = np.fft.fftfreq(len(field_y), d=dy) #* 2 * np.pi
    KX, KY = np.meshgrid(kx, ky, indexing='ij')

    pwr=12 #super gaussian power
    # k_filter_x = 0.8
    # k_filter_y = 15
    mask = np.exp(- np.sqrt((KX / k_filter_x)**2 + (KY / k_filter_y)**2)**pwr)
    plt.figure()
    plt.imshow(np.fft.fftshift(mask).T, extent=(kx.min(), kx.max(), ky.min(), ky.max()), origin='lower')
    plt.colorbar()

    data_fft  = np.fft.fft2(data, axes=(0,1))   # shape (Nx, Ny)
    data_fft_shift = np.fft.fftshift(data_fft)
    plt.figure()
    plt.imshow(np.log10(np.abs(data_fft_shift.T)), extent=(kx.min(), kx.max(), ky.min(), ky.max()), origin='lower')
    plt.colorbar()
    
    Diag_data = np.real(np.fft.ifft2(data_fft* mask[:,:]))
    
    return kx, ky, data, data_fft, Diag_data


In [228]:
kx3, ky3, Bx3, Bx3_fft, Bx3_masked = fft_field_analysis2(S3, 0, "Bx", 'z', Lz/2., -1, 0.7, 15)


[ 0.          0.08002468  0.16004936  0.24007404  0.32009872  0.4001234
  0.48014808  0.56017276  0.64019744  0.72022212  0.8002468   0.88027148
  0.96029616  1.04032084  1.12034552  1.2003702   1.28039488  1.36041956
  1.44044424  1.52046892  1.6004936   1.68051828  1.76054296  1.84056764
  1.92059232  2.000617    2.08064168  2.16066636  2.24069104  2.32071572
  2.4007404   2.48076508  2.56078976  2.64081444  2.72083912  2.80086381
  2.88088849  2.96091317  3.04093785  3.12096253  3.20098721  3.28101189
  3.36103657  3.44106125  3.52108593  3.60111061  3.68113529  3.76115997
  3.84118465  3.92120933  4.00123401  4.08125869  4.16128337  4.24130805
  4.32133273  4.40135741  4.48138209  4.56140677  4.64143145  4.72145613
  4.80148081  4.88150549  4.96153017  5.04155485  5.12157953  5.20160421
  5.28162889  5.36165357  5.44167825  5.52170293  5.60172761  5.68175229
  5.76177697  5.84180165  5.92182633  6.00185101  6.08187569  6.16190037
  6.24192505  6.32194973  6.40197441  6.48199909  6.

In [197]:
plt.imshow(np.log10(np.abs(Bx3_fft.T)), 
           extent=(kx0.min(), kx0.max(), ky0.min(), ky0.max()), 
           origin='lower', 
        #    vmin=-5,
        #    vmax=-1,
           aspect='auto')
# plt.colorbar(label='Log10 |Bx FFT|')
# plt.xlim([-2,2])
# plt.ylim([-2,2])
plt.xlabel('kx (1/um)')
plt.ylabel('ky (1/um)')
# plt.title('FFT of Bx field at z=Lz/2, time=133.10 fs')
plt.tight_layout()

In [ ]:
def fft_field_analysis(
    S,
    diag_num,
    field_name,
    slice_direction,
    slice_position,
    shift,
    timestep,
    k0x,
    k0y,
    wx,
    wy,
):
    """
    Perform FFT analysis on a given field from the simulation data.

    Parameters:
    S : happi.Simulation
        The simulation object.
    diag_num : int
        The diagnostic number of the field.
    field_name : str
        The name of the field to analyze.
    slice_direction : str
        e.g., 'x', 'y', 'z', etc.
    slice_position : float
        The position to slice the field (e.g., Lz/2).
    shift : bool
        Whether to shift the sample region to avoid boundary condition issues.
    timestep : int
        The timestep at which to perform the analysis.
    k_filter_x, k_filter_y : float
        The mask to filter out the signals. Laser is 1.

    Returns:
    kx : np.ndarray
        The wave numbers in the x-direction.
    ky : np.ndarray
        The wave numbers in the y-direction.
    spectrum : np.ndarray
        The FFT spectrum of the field.
    """

    twopi = 2.0 * np.pi
    var0_time = S.Probe(diag_num, field_name, units=["fs"]).getTimesteps()
    print("timesteps of var : ", var0_time)
    print("num. of timesteps: ", var0_time.size)

    var0 = S.Probe(diag_num, field_name, units=["fs"]).getData(
        timestep=var0_time[timestep]
    )[0]
    print("shape of var: ", var0.shape)

    var0_time_real = S.Probe(diag_num, field_name, units=["fs"]).getTimes()
    print("time of var : {:.2f} fs".format(var0_time_real[timestep]))

    field = S.Probe(
        diag_num,
        field_name,
        subset={slice_direction: slice_position},
        units=["um", "fs", "1e9 Gauss"],
        vsym=True,
    )

    data = np.array(field.getData()[timestep])

    if shift == True:
        nx = data.shape[0]
        ny = data.shape[1]
        sf = 0  # shift of cells to get rid of boundary condition
        print("note that the sample region is shifted to get rid of the BC issue")
        print("No. of cells being shifted, ", sf)
        #         print(nx, ny)
        data = data[sf : (nx - sf), sf : (ny - sf)]

    field_x = np.array(field.getAxis("axis1"))
    field_y = np.array(field.getAxis("axis2"))

    Nx = data.shape[0]
    Ny = data.shape[1]

    dx = (field_x[1] - field_x[0])[0]
    dy = (field_y[1] - field_y[0])[1]

    kx = np.fft.fftfreq(Nx, d=dx) * 2 * np.pi  # spatial freq extents
    ky = np.fft.fftfreq(Ny, d=dy) * 2 * np.pi
    KX, KY = np.meshgrid(kx, ky, indexing="ij")

    # This is another way to construct kx, ky grids -- but somehow doesn't work as expected
    # kx=np.linspace(-Nx/2,Nx/2-1,Nx)/((Nx)*dx)  ## -1? min&max??
    # ky=np.linspace(-Ny/2,Ny/2-1,Ny)/((Ny)*dy)
    # KX, KY = np.meshgrid(twopi*kx, twopi*ky, indexing='ij')

    # construct mask
    # super gaussian mask
    pwr = 12  # super gaussian power
    mask = np.exp(-(np.sqrt(((KX - k0x) / wx) ** 2 + ((KY - k0y) / wy) ** 2) ** pwr))
    # a quick check of the mask
    # plt.imshow(mask.T, extent=(kx.min(), kx.max(), ky.min(), ky.max()), origin="lower")

    # another notch filter mask, which is not used for now
    def notch_mask(KX, KY, k0x, k0y, wx, wy, p=12):
        # four symmetric notches around ±k0x and ±k0y
        notch_1 = np.exp(
            -((((KX - k0x) / wx) ** 2 + ((KY - k0y) / wy) ** 2) ** (p / 2))
        )
        notch_2 = np.exp(
            -((((KX + k0x) / wx) ** 2 + ((KY - k0y) / wy) ** 2) ** (p / 2))
        )
        notch_3 = np.exp(
            -((((KX - k0x) / wx) ** 2 + ((KY + k0y) / wy) ** 2) ** (p / 2))
        )
        notch_4 = np.exp(
            -((((KX + k0x) / wx) ** 2 + ((KY + k0y) / wy) ** 2) ** (p / 2))
        )
        return (1 - notch_1) * (1 - notch_2) * (1 - notch_3) * (1 - notch_4)

    # mask = notch_mask(KX, KY, k0x, k0y, wx, wy, p=12)
    # plt.imshow(mask.T, extent=(kx.min(), kx.max(), ky.min(), ky.max()), origin='lower')
    # plt.xlim([0,1])
    # plt.ylim([0,1])

    data_fft = np.fft.fft2(data, axes=(0, 1))  # shape (Nx, Ny)
    Diag_data = np.real(np.fft.ifft2(data_fft * mask[:, :]))

    # data_fft  = np.fft.fft2(data, axes=(0,1))/Nx/Ny*2   # shape (Nx, Ny)
    data_fft_shift = np.fft.fftshift(data_fft)
    data_fft_shift_masked = data_fft_shift * mask[:, :]
    Diag_data_shift_back = np.fft.ifftshift(data_fft_shift * mask[:, :])
    # Diag_data = np.real(np.fft.ifft2(Diag_data_shift_back))
    # Diag_data = np.real(np.fft.ifft2(data_fft_shift* mask[:,:]))

    return kx, ky, data, data_fft, data_fft_shift, data_fft_shift_masked, Diag_data


k0x = 0.0
k0y = 0.0
wx = 0.5
wy = 2.0

In [53]:
kx3, ky3, Bx3, Bx3_fft, Bx3_fft_shifted, Bx3_fft_shifted_masked, Bx3_masked = (
    fft_field_analysis(S3, 0, "Bx", "z", Lz / 2.0, False, -1, k0x, k0y, wx, wy)
)

timesteps of var :  [   0.  182.  364.  546.  728.  910. 1092. 1274. 1456. 1638. 1820. 2002.
 2184. 2366. 2548. 2730. 2912. 3094. 3276.]
num. of timesteps:  19
shape of var:  (1000, 1220)
time of var : 299.47 fs


In [54]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(figsize=(6, 5))

im = ax.imshow(
    # Bx3_masked.T,
    Bx3.T,
    origin="lower",
    extent=[0, Lx, 0, Ly],
    vmin=-20,
    vmax=20,
    aspect="equal",
    cmap="bwr",
)

# Make a colorbar axis that exactly matches the height of `ax`
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)  # width & gap
cb = fig.colorbar(im, cax=cax, label="Bx (GG)")

ax.set_xlabel("x (um)")
ax.set_ylabel("y (um)")
fig.tight_layout()
# plt.show()

# plt.tight_layout()
# plt.savefig("/Users/yao/Desktop/Bx_a0_350.png", dpi=300, bbox_inches="tight")

In [55]:
fig, ax = plt.subplots(figsize=(6, 5))

im = ax.imshow(
    Bx3_masked.T,
    # Bx3.T,
    origin="lower",
    extent=[0, Lx, 0, Ly],
    vmin=-3,
    vmax=3,
    aspect="equal",
    cmap="bwr",
)

# Make a colorbar axis that exactly matches the height of `ax`
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)  # width & gap
cb = fig.colorbar(im, cax=cax, label="Bx (GG)")

ax.set_xlabel("x (um)")
ax.set_ylabel("y (um)")
fig.tight_layout()
plt.show()

# plt.tight_layout()
# plt.savefig("/Users/yao/Desktop/Bx_a0_350_fft.png", dpi=300, bbox_inches="tight")

In [65]:
plt.figure()
plt.imshow(
    # np.log10(np.abs(Bx3_fft_shifted.T)),
    np.abs(Bx3_fft_shifted_masked.T),
    # extent=(kx3.min(), kx3.max(), ky3.min(), ky3.max()),
    origin="lower",
    #    vmin=-3,
    #    vmax=1,
    aspect="auto",
)
plt.colorbar(label="|Bx FFT|")
# plt.xlim([-1, 1])
# plt.ylim([-1, 1])
plt.xlabel("kx (1/um)")
plt.ylabel("ky (1/um)")
# plt.title('FFT of Bx field at z=Lz/2, time=133.10 fs')
plt.tight_layout()
# plt.savefig("/Users/yao/Desktop/Bx_a0_350_mask.png", dpi=300, bbox_inches="tight")

In [57]:
Bx3_fft_shifted_masked.max(), Bx3_fft_shifted_masked.min()

((64.20315925910772+21.01317994052597j),
 (-53.96698845412195+23.74162490949556j))

In [63]:
np.abs(Bx3_fft_shifted_masked.T)

array([[35.59343995, 40.04514121, 24.24180822, ..., 19.06834488,
        24.24180822, 40.04514121],
       [37.52986338, 22.147452  , 15.33421812, ..., 29.37797291,
        24.63430928, 29.87364758],
       [34.6738643 , 29.44556647, 43.70233094, ...,  5.5915253 ,
        12.43389108, 14.61428921],
       ...,
       [42.97185581, 12.52841543, 15.31369576, ..., 51.13585717,
        32.03295071, 20.09739133],
       [34.6738643 , 14.61428921, 12.43389108, ..., 58.95846501,
        43.70233094, 29.44556647],
       [37.52986338, 29.87364758, 24.63430928, ..., 28.03642386,
        15.33421812, 22.147452  ]])